In [1]:
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
getattr(ssl, '_create_unverified_context', None)):
    ssl._create_default_https_context = ssl._create_unverified_context
import datetime as dt
import pandas as pd
from sqlalchemy import create_engine
from config import pga_admin

In [2]:
# SOURCE:  National Geophysical Data Center / World Data Service (NGDC/WDS): 
#          Significant Earthquake Database. National Geophysical Data Center, NOAA. doi:10.7289/V5TD9V7K
#          Significant Earthquake Search - total effects displayed, sorted by Date
quake_df = pd.read_csv("NGDC_dataset.csv")

In [3]:
quake_df

,I_D,FLAG_TSUNAMI,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND,FOCAL_DEPTH,EQ_PRIMARY,...,TOTAL_MISSING,TOTAL_MISSING_DESCRIPTION,TOTAL_INJURIES,TOTAL_INJURIES_DESCRIPTION,TOTAL_DAMAGE_MILLIONS_DOLLARS,TOTAL_DAMAGE_DESCRIPTION,TOTAL_HOUSES_DESTROYED,TOTAL_HOUSES_DESTROYED_DESCRIPTION,TOTAL_HOUSES_DAMAGED,TOTAL_HOUSES_DAMAGED_DESCRIPTION
0,4333,NaN,1965,1,1,21.0,38.0,29.2,10.0,5.5,...,NaN,NaN,350.0,3.0,2.00,2.0,2500.0,4.0,NaN,NaN
1,8074,NaN,1965,1,12,17.0,18.0,9,10.0,5.5,...,NaN,NaN,NaN,NaN,NaN,2.0,NaN,3.0,NaN,NaN
2,4334,Tsu,1965,1,24,0.0,11.0,12.1,6.0,7.6,...,NaN,NaN,NaN,NaN,NaN,2.0,3000.0,4.0,NaN,NaN
3,4337,Tsu,1965,2,4,5.0,1.0,21.6,36.0,8.7,...,NaN,NaN,NaN,NaN,0.01,1.0,NaN,NaN,NaN,NaN
4,6411,Tsu,1965,2,23,22.0,11.0,46,36.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1471,10380,NaN,2016,12,8,5.0,15.0,6,18.0,6.0,...,NaN,NaN,NaN,NaN,135.00,4.0,NaN,NaN,NaN,NaN
1472,10218,Tsu,2016,12,8,17.0,38.0,28,40.0,7.8,...,NaN,NaN,NaN,NaN,NaN,1.0,191.0,3.0,126.0,3.0
1473,10219,Tsu,2016,12,9,19.0,11.0,24,20.0,6.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1474,10220,Tsu,2016,12,17,10.0,51.0,5,95.0,7.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
quake_df2 = quake_df[['FLAG_TSUNAMI', 'YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE', 'SECOND', 'FOCAL_DEPTH', 'EQ_PRIMARY', 
          'COUNTRY', 'STATE', 'LOCATION_NAME', 'LATITUDE', 'LONGITUDE', 'REGION_CODE', 'DEATHS', 'INJURIES',
          'DAMAGE_MILLIONS_DOLLARS', 'HOUSES_DESTROYED', 'HOUSES_DAMAGED', 'TOTAL_DEATHS', 'TOTAL_INJURIES',
          'TOTAL_DAMAGE_MILLIONS_DOLLARS', 'TOTAL_HOUSES_DESTROYED', 'TOTAL_HOUSES_DAMAGED']]

In [5]:
# Clear all NaNs
quake_df2 = quake_df2.fillna(0)
quake_df2.columns

Index(['FLAG_TSUNAMI', 'YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE', 'SECOND',
       'FOCAL_DEPTH', 'EQ_PRIMARY', 'COUNTRY', 'STATE', 'LOCATION_NAME',
       'LATITUDE', 'LONGITUDE', 'REGION_CODE', 'DEATHS', 'INJURIES',
       'DAMAGE_MILLIONS_DOLLARS', 'HOUSES_DESTROYED', 'HOUSES_DAMAGED',
       'TOTAL_DEATHS', 'TOTAL_INJURIES', 'TOTAL_DAMAGE_MILLIONS_DOLLARS',
       'TOTAL_HOUSES_DESTROYED', 'TOTAL_HOUSES_DAMAGED'],
      dtype='object')

In [6]:
# Way more info than necessary.
# Reshaping this data into a new DF with renamed headers and fewer columns
quake_df3 = quake_df2.rename(columns={'FLAG_TSUNAMI':'Tsunami','YEAR': 'Y', 'MONTH': 'Mo','DAY': 'D', 'HOUR': 'H',
                          'MINUTE': 'Mi','SECOND': 'S','FOCAL_DEPTH': 'Depth', 'EQ_PRIMARY': 'Magnitude',
                          'COUNTRY': 'Country', 'STATE': 'State', 'LOCATION_NAME': 'Location', 'LATITUDE': 'Latitude',
                          'LONGITUDE': 'Longitude', 'REGION_CODE': 'Region', 'DEATHS': 'Deaths', 
                          'INJURIES': 'Injuries','DAMAGE_MILLIONS_DOLLARS': 'Damages_Millions', 
                          'HOUSES_DESTROYED': 'Houses_Destroyed', 'HOUSES_DAMAGED': 'Houses_Damaged',
                          'TOTAL_DEATHS': 'Total_Deaths', 'TOTAL_INJURIES': 'Total_Injuries', 
                          'TOTAL_DAMAGE_MILLIONS_DOLLARS': 'Total_Damages_Millions',
                          'TOTAL_HOUSES_DESTROYED': 'Tot_Houses_Destroyed',
                          'TOTAL_HOUSES_DAMAGED': 'Tot_Houses_Damaged'})

quake_df3

,Tsunami,Y,Mo,D,H,Mi,S,Depth,Magnitude,Country,...,Deaths,Injuries,Damages_Millions,Houses_Destroyed,Houses_Damaged,Total_Deaths,Total_Injuries,Total_Damages_Millions,Tot_Houses_Destroyed,Tot_Houses_Damaged
0,0,1965,1,1,21.0,38.0,29.2,10.0,5.5,ALGERIA,...,4.0,350.0,2.0,25000.0,0.0,4.0,350.0,2.00,2500.0,0.0
1,0,1965,1,12,17.0,18.0,9,10.0,5.5,CHINA,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
2,Tsu,1965,1,24,0.0,11.0,12.1,6.0,7.6,INDONESIA,...,71.0,0.0,0.0,3000.0,0.0,71.0,0.0,0.00,3000.0,0.0
3,Tsu,1965,2,4,5.0,1.0,21.6,36.0,8.7,USA,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0
4,Tsu,1965,2,23,22.0,11.0,46,36.0,7.0,CHILE,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1471,0,2016,12,8,5.0,15.0,6,18.0,6.0,CHINA,...,1.0,0.0,135.0,0.0,0.0,1.0,0.0,135.00,0.0,0.0
1472,Tsu,2016,12,8,17.0,38.0,28,40.0,7.8,SOLOMON ISLANDS,...,1.0,0.0,0.0,191.0,126.0,1.0,0.0,0.00,191.0,126.0
1473,Tsu,2016,12,9,19.0,11.0,24,20.0,6.9,SOLOMON ISLANDS,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
1474,Tsu,2016,12,17,10.0,51.0,5,95.0,7.9,PAPUA NEW GUINEA,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0


In [7]:
# The Year, Month, Day, Hour, Min, Second columns are too granular - just need MM/DD/YYYY in a "Date" column

# Converting Year, Month, and Day to string...
quake_df3["Y"] = quake_df3["Y"].astype(str)
quake_df3["Mo"] = quake_df3["Mo"].astype(str)
quake_df3["D"] = quake_df3["D"].astype(str)

# ...allows me to merge them into a single column with the .agg func
quake_df3['Date'] = quake_df3[['Mo', 'D', 'Y']].agg('-'.join, axis=1)

In [8]:
# Reform the data into v.4, removing extra date columns, extraneous items, & Proper Caps (for SQL table)
# 28 cols removed!
quake_df4 = quake_df3[['Date','Tsunami', 'Depth', 'Magnitude',
       'Country', 'State', 'Location', 'Latitude', 'Longitude', 'Deaths',
       'Injuries', 'Damages_Millions', 'Houses_Destroyed', 'Houses_Damaged',
       'Total_Deaths', 'Total_Injuries', 'Total_Damages_Millions',
       'Tot_Houses_Destroyed', 'Tot_Houses_Damaged']]

In [9]:
# Cleaning up the Tsunami (a simple Yes or No) and State cols
quake_df5 = quake_df4.replace({'Tsunami': 0}, 'Y')
quake_df6 = quake_df5.replace({'Tsunami': 'Tsu'}, 'N')
quake_df7 = quake_df6.replace({'State': 0}, 'n/a')
quake_df7

,Date,Tsunami,Depth,Magnitude,Country,State,Location,Latitude,Longitude,Deaths,Injuries,Damages_Millions,Houses_Destroyed,Houses_Damaged,Total_Deaths,Total_Injuries,Total_Damages_Millions,Tot_Houses_Destroyed,Tot_Houses_Damaged
0,1-1-1965,Y,10.0,5.5,ALGERIA,n/a,ALGERIA: M'SILA,35.700,4.400,4.0,350.0,2.0,25000.0,0.0,4.0,350.0,2.00,2500.0,0.0
1,1-12-1965,Y,10.0,5.5,CHINA,n/a,CHINA: SHANXI PROVINCE,35.000,111.630,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
2,1-24-1965,N,6.0,7.6,INDONESIA,n/a,INDONESIA: SANANA ISLAND,-2.400,126.100,71.0,0.0,0.0,3000.0,0.0,71.0,0.0,0.00,3000.0,0.0
3,2-4-1965,N,36.0,8.7,USA,AK,ALASKA: ALEUTIAN ISLANDS: RAT ISLANDS,51.290,178.550,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.0
4,2-23-1965,N,36.0,7.0,CHILE,n/a,CHILE: NORTHERN,-25.670,-70.630,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1471,12-8-2016,Y,18.0,6.0,CHINA,n/a,CHINA: N. XINJIANG: URUMQI,43.823,86.345,1.0,0.0,135.0,0.0,0.0,1.0,0.0,135.00,0.0,0.0
1472,12-8-2016,N,40.0,7.8,SOLOMON ISLANDS,n/a,SOLOMON ISLANDS,-10.681,161.327,1.0,0.0,0.0,191.0,126.0,1.0,0.0,0.00,191.0,126.0
1473,12-9-2016,N,20.0,6.9,SOLOMON ISLANDS,n/a,SOLOMON ISLANDS,-10.749,161.132,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
1474,12-17-2016,N,95.0,7.9,PAPUA NEW GUINEA,n/a,PAPUA NEW GUINEA: NEW BRITAIN NEW IRELAND,-4.505,153.522,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0


In [10]:
# Connect to the empty table 'etl_project' in PGAdmin4
rds_connection_string = pga_admin
engine = create_engine(f'postgresql://{rds_connection_string}')

In [11]:
# View Table Name in PGAdmin
engine.table_names()

['quake_table']

In [12]:
#Load quake_df7 data into the SQL table 
quake_df7.to_sql(name='quake_table', con=engine, if_exists='append', index=False)